# A Guide to TF Layers: Building a Convolutional Neural Network

## TensorFlow의 Layers module 은 높은 수준의 API를 제공한다.
### fully connected layer나 convolutional layer또는 활성함수,드롭아웃등을 제공한다.

In [12]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# Imports
import numpy as np
import tensorflow as tf

from tensorflow.contrib import learn
from tensorflow.contrib.learn.python.learn.estimators import model_fn as model_fn_lib

tf.logging.set_verbosity(tf.logging.INFO)

# Our application logic will be added here

In [13]:
def cnn_model_fn(features, labels, mode):
    """Model function for CNN."""
    # Input Layer
    input_layer = tf.reshape(features, [-1, 28, 28, 1])

  # Convolutional Layer #1
    conv1 = tf.layers.conv2d(
        inputs=input_layer,
        filters=32,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)

  # Pooling Layer #1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2 and Pooling Layer #2
    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=64,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Dense Layer
    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    dropout = tf.layers.dropout(
        inputs=dense, rate=0.4, training=mode == learn.ModeKeys.TRAIN)

    # Logits Layer
    logits = tf.layers.dense(inputs=dropout, units=10)

    loss = None
    train_op = None

  # Calculate Loss (for both TRAIN and EVAL modes)
    if mode != learn.ModeKeys.INFER:
        onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
        loss = tf.losses.softmax_cross_entropy(onehot_labels=onehot_labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == learn.ModeKeys.TRAIN:
        train_op = tf.contrib.layers.optimize_loss(loss=loss, global_step=tf.contrib.framework.get_global_step(),
          learning_rate=0.001,
          optimizer="SGD")

    # Generate Predictions
    predictions = {
        "classes": tf.argmax(
            input=logits, axis=1),
        "probabilities": tf.nn.softmax(
            logits, name="softmax_tensor")
    }

    # Return a ModelFnOps object
    return model_fn_lib.ModelFnOps(
        mode=mode, predictions=predictions, loss=loss, train_op=train_op)

In [17]:
def main(unused_argv):
  # Load training and eval data
    mnist = learn.datasets.load_dataset("mnist")
    train_data = mnist.train.images # Returns np.array
    train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
    eval_data = mnist.test.images # Returns np.array
    eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)
    # Set up logging for predictions
    tensors_to_log = {"probabilities": "softmax_tensor"}
    logging_hook = tf.train.LoggingTensorHook(
      tensors=tensors_to_log, every_n_iter=50)
    # Create the Estimator
    mnist_classifier = learn.Estimator(
        model_fn=cnn_model_fn, model_dir="/tmp/mnist_convnet_model")
    # Train the model
    mnist_classifier.fit(
      x=train_data,
      y=train_labels,
      batch_size=100,
      steps=2000,
      monitors=[logging_hook])
    # Configure the accuracy metric for evaluation
    metrics = {
      "accuracy":
          learn.MetricSpec(
              metric_fn=tf.metrics.accuracy, prediction_key="classes"),
    }
    # Evaluate the model and print results
    eval_results = mnist_classifier.evaluate(
      x=eval_data, y=eval_labels, metrics=metrics)
    print(eval_results)

In [19]:
tf.app.run()

Extracting MNIST-data\train-images-idx3-ubyte.gz
Extracting MNIST-data\train-labels-idx1-ubyte.gz
Extracting MNIST-data\t10k-images-idx3-ubyte.gz
Extracting MNIST-data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimato

D:\python3_5anaconda\lib\site-packages\tensorflow\python\util\deprecation.py:248: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  equality = a == b


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /tmp/mnist_convnet_model\model.ckpt-1
INFO:tensorflow:Saving checkpoints for 2 into /tmp/mnist_convnet_model\model.ckpt.
INFO:tensorflow:loss = 2.29625, step = 2
INFO:tensorflow:probabilities = [[ 0.10107068  0.0817854   0.10115192  0.10957383  0.10864981  0.11591488
   0.09629025  0.09235857  0.09940265  0.09380201]
 [ 0.12051555  0.08240568  0.11267658  0.09782542  0.0989892   0.10448267
   0.10609802  0.09831707  0.08953912  0.08915062]
 [ 0.10626867  0.08774996  0.11013414  0.10004747  0.11427978  0.09676079
   0.1039373   0.08802529  0.09823348  0.09456304]
 [ 0.10604544  0.08815608  0.09846032  0.12140784  0.10248565  0.09731533
   0.10470735  0.10103702  0.09926398  0.08112105]
 [ 0.11678673  0.08678851  0.10376825  0.10627253  0.11393198  0.09273189
   0.09909867  0.07511468  0.10113598  0.10437074]
 [ 0.1110404   0.08075479  0.10913061  0.09421501  0.1097477   0.10378144
   0.09995303  0.0948

INFO:tensorflow:probabilities = [[ 0.10911719  0.08005448  0.11160397  0.10468394  0.10433508  0.10080764
   0.09584061  0.10371756  0.09373406  0.09610544]
 [ 0.11074068  0.07619704  0.09870852  0.10628877  0.11489778  0.09698656
   0.1145044   0.09327716  0.09922457  0.08917456]
 [ 0.10399359  0.08691703  0.1087484   0.0974014   0.1072491   0.10565531
   0.10071088  0.09411044  0.09942856  0.09578517]
 [ 0.10785005  0.0905123   0.10762646  0.1084692   0.11508411  0.10928847
   0.08157967  0.0937473   0.09644306  0.08939933]
 [ 0.12198585  0.08638961  0.10978967  0.10806529  0.1056108   0.10484693
   0.08708421  0.08634594  0.10341663  0.08646509]
 [ 0.11505824  0.08262707  0.10648964  0.10113452  0.10959585  0.10404258
   0.09761173  0.09020039  0.08996775  0.10327218]
 [ 0.11028706  0.08314144  0.10722141  0.09582632  0.11599967  0.10558875
   0.09911332  0.10025759  0.09023718  0.09232736]
 [ 0.1077762   0.08433463  0.1050493   0.10306532  0.10697389  0.10082994
   0.10198509  0.09

INFO:tensorflow:global_step/sec: 2.28342
INFO:tensorflow:loss = 2.27185, step = 102 (43.799 sec)
INFO:tensorflow:probabilities = [[ 0.11714502  0.08069782  0.10534849  0.10455307  0.10659848  0.08670793
   0.10729543  0.09916899  0.08823276  0.10425197]
 [ 0.11816974  0.07962741  0.10685984  0.11244573  0.11082789  0.10774786
   0.08743908  0.08793327  0.09774234  0.09120686]
 [ 0.10536236  0.08645984  0.1042292   0.11149406  0.10874574  0.10483174
   0.0921268   0.08192777  0.1022257   0.10259677]
 [ 0.10052814  0.09250397  0.10173061  0.10601304  0.11215663  0.10064411
   0.09908246  0.09858247  0.09655515  0.09220337]
 [ 0.10829312  0.07674204  0.12063131  0.11310815  0.11381988  0.09541076
   0.09657895  0.08907418  0.09681335  0.08952824]
 [ 0.1065627   0.08006817  0.1064077   0.10432101  0.09586153  0.11111648
   0.10530987  0.10535588  0.09714577  0.08785088]
 [ 0.12267664  0.08314807  0.10882454  0.10109533  0.10064507  0.09970707
   0.10564241  0.09923095  0.09018178  0.088848

...

INFO:tensorflow:global_step/sec: 2.47843
INFO:tensorflow:loss = 2.24775, step = 302 (40.349 sec)
INFO:tensorflow:probabilities = [[ 0.115182    0.07594437  0.11499885  0.10252077  0.09205447  0.09318147
   0.10439368  0.11069294  0.09526111  0.0957704 ]
 [ 0.10454957  0.09387843  0.10510774  0.1097485   0.10196682  0.10190758
   0.10734744  0.0875723   0.09440812  0.09351348]
 [ 0.10741938  0.08162434  0.1005459   0.10564239  0.10887089  0.09782902
   0.09843037  0.10037221  0.09574868  0.10351684]
 [ 0.10858147  0.09875119  0.1019332   0.10095811  0.09602166  0.10779381
   0.09914896  0.09291591  0.10126624  0.09262937]
 [ 0.11560362  0.09080841  0.10935948  0.09252293  0.10408007  0.09665634
   0.10099725  0.10237695  0.08825959  0.0993353 ]
 [ 0.11290498  0.08281521  0.11289638  0.10305073  0.0941571   0.11083455
   0.0959812   0.0908363   0.09532386  0.10119969]
 [ 0.12281052  0.07744848  0.11332749  0.09803206  0.10692144  0.11202735
   0.08907759  0.0915657   0.09984218  0.088947

INFO:tensorflow:probabilities = [[ 0.09298591  0.09551658  0.1134096   0.11478595  0.10020637  0.09923984
   0.08685784  0.09914206  0.10137535  0.09648053]
 [ 0.10463183  0.07903899  0.10805308  0.12636729  0.1080108   0.10336995
   0.09704548  0.0854058   0.09534568  0.09273109]
 [ 0.10700496  0.08649512  0.12159866  0.10696139  0.10278095  0.10509198
   0.11093167  0.08216741  0.09209789  0.08486989]
 [ 0.13488716  0.07364075  0.10835534  0.10121267  0.09959019  0.1079729
   0.0866318   0.11177402  0.09656622  0.07936902]
 [ 0.10891239  0.09432661  0.10225426  0.10709199  0.10273368  0.1063862
   0.0893914   0.09551181  0.10049474  0.09289701]
 [ 0.11245862  0.07567996  0.09743943  0.12907663  0.09669312  0.10151596
   0.09985027  0.08974933  0.11007936  0.08745719]
 [ 0.10409733  0.10255331  0.09366668  0.10953262  0.10153501  0.09500965
   0.09784921  0.09959775  0.09887635  0.0972821 ]
 [ 0.10914255  0.09521258  0.09981684  0.10717206  0.10119824  0.10886325
   0.09556501  0.0984

INFO:tensorflow:global_step/sec: 2.47759
INFO:tensorflow:loss = 2.26065, step = 402 (40.359 sec)
INFO:tensorflow:probabilities = [[ 0.10360406  0.09183492  0.10450286  0.11024558  0.09357046  0.10497981
   0.10712723  0.08768789  0.10363515  0.09281203]
 [ 0.11643197  0.08932894  0.10680746  0.11164019  0.1028152   0.10958122
   0.08492349  0.09982184  0.09157081  0.08707889]
 [ 0.1133794   0.0793969   0.09487202  0.11542024  0.10012411  0.10103599
   0.10190124  0.09602764  0.09657614  0.10126639]
 [ 0.10540824  0.10277105  0.09880684  0.10305436  0.0996546   0.09562082
   0.1032619   0.09794902  0.10371745  0.08975574]
 [ 0.10740427  0.08261593  0.11555305  0.10882144  0.09677801  0.11028048
   0.0991633   0.09748843  0.09210807  0.08978712]
 [ 0.10771577  0.09330824  0.09934447  0.10592273  0.09996443  0.09875899
   0.09125758  0.11336288  0.0975363   0.09282867]
 [ 0.11345012  0.08200306  0.10690896  0.10637103  0.09843598  0.09270301
   0.11344983  0.08663961  0.10497914  0.095059

INFO:tensorflow:probabilities = [[ 0.10072584  0.08711998  0.09554149  0.10393944  0.11416572  0.101032
   0.10019447  0.08727122  0.10543864  0.10457125]
 [ 0.10435463  0.07723349  0.12013328  0.13415053  0.10522331  0.10552333
   0.09123032  0.08502698  0.09810322  0.07902089]
 [ 0.10389302  0.09518972  0.10471539  0.10245583  0.0955711   0.09794832
   0.11121085  0.08939447  0.0997103   0.09991091]
 [ 0.11736086  0.0869962   0.11210161  0.10462481  0.10114858  0.11835761
   0.09628765  0.09029459  0.10040417  0.07242393]
 [ 0.1335905   0.07508241  0.09268012  0.11304377  0.10317035  0.11036947
   0.09316243  0.10060708  0.09225573  0.08603811]
 [ 0.11346037  0.08098173  0.11088333  0.11375167  0.10388453  0.1087519
   0.08593889  0.07951526  0.10641269  0.09641959]
 [ 0.12913942  0.07597707  0.09222096  0.11285469  0.10571051  0.11347064
   0.10830434  0.08487758  0.07631961  0.10112522]
 [ 0.11116818  0.09047269  0.10451819  0.0987794   0.0985309   0.10288219
   0.09576631  0.09432

KeyboardInterrupt: 